# Class 6: Image Smoothing and Convolutional Filters

## Preliminaries

Run the cell below to download the course library and class resources.

In [ ]:
import gdown

gdown.download(id='1SzvuBYIZ407c9eOChXD48NG94v7azJby')
gdown.download(id='1K8nNMjlhWPTgL4OyozYCLaduGmvx1WO-')

!unzip -o '06.zip'
!rm '06.zip'

Run the cell below to import the class modules.

If you get import warnings, try using **`Ctrl+M .`** to restart the kernel. *(notice there is a dot there)*

In [ ]:
import numpy as np
import cv2 as cv

from sdx import *

## Choosing an image

Image smoothing is a process with inevitable tradeoff: the more noise you reduce, the more details you lose. To illustrate this tradeoff, the idea of this notebook is testing your code with 18 different images: the combinations of 6 different sources and 3 different severities of noise.

The options of source are: `smash`, `atletica`, `consulting`, `insper`, `informatica`, and `harvard`. Half of these sources are logos and the other half are photos. Some have more uniform regions and some have more small details.



In [ ]:
SOURCE = 'insper'

The options of severity are `8`, `16`, and `32`. The number represents the approximate percentage of levels that are slightly below or above.

In [ ]:
SEVERITY = 8

## Loading and displaying the image

Like in Class 4, we will use the `asfloat=True` parameter to read the image as a 64-bit float array.

In [ ]:
input = cv_grayread(f'{SOURCE}-{SEVERITY}.png', asfloat=True)

cv_imshow(input)

## Activity 0: simple smoothing

Read and understand the code below. All the details. They might be useful.

In [ ]:
def simple_smooth(input, size):
    height, width = input.shape

    radius = size // 2

    total = size ** 2

    output = np.empty((height - 2 * radius, width - 2 * radius))

    for oy, iy in enumerate(range(radius, height - radius)):
        for ox, ix in enumerate(range(radius, width - radius)):
             sum = 0

             for dy in range(-radius, radius + 1):
                 for dx in range(-radius, radius + 1):
                     sum += input[iy + dy, ix + dx]

             output[oy, ox] = sum / total

    return output

You can use the image below as a benchmark for the next activities.

In [ ]:
cv_imshow(simple_smooth(input, 3))

## Activity 1: simple kernel

Write a function that receives a positive integer `size` and returns a `size`x`size` NumPy array where each element is `1 / size ** 2`.

Try to avoid using loops. Research for useful NumPy array constructors.

In [ ]:
def simple_kernel(size):
    return np.ones((size, size)) * 1/(size**2)

If your function is working, the next cell should display the following result.

```
array([[0.11111111, 0.11111111, 0.11111111],
       [0.11111111, 0.11111111, 0.11111111],
       [0.11111111, 0.11111111, 0.11111111]])
```

In [ ]:
simple_kernel(3)

## Activity 2: convolutional smoothing

This time, *do* use loops. The `scipy` library has the `signal.convolve2d` function, but do *not* use it or any other already implemented solution. In this particular activity, it's important to think about the implementation.

In [ ]:
def convolution_smooth(input, kernel):
    size = kernel.shape[0]
    padding = size//2
    output = np.zeros(input.shape)
    for i in range(padding, input.shape[0] - padding):
      for j in range(padding, input.shape[1] - padding):
        output[i,j] = np.sum(input[i-padding:i+padding+1, j-padding:j+padding+1] * kernel)
    return output

Validate your code by comparing the output below with the output from Activity 0.

In [ ]:
kernel = simple_kernel(3)

cv_imshow(convolution_smooth(input, kernel))

## Challenge 1: gaussian kernel

Write a function that receives a positive integer `size` and a float `sigma` and returns a `size`x`size` NumPy array where each element is given by the formula below. **However, when applying this formula, you must assume that the origin is at the center of the array**.

$$\frac{e^{-(x^2+y^2)/2\sigma^2}}{2\pi\sigma^2}$$

Again, please avoid already implemented solutions this time.

In [ ]:
from math import pi, e

def gaussian_kernel(size, sigma):
    kernel = np.zeros([size, size])
    for i in range(size):
      for j in range(size):
        x = i - size/2
        y = j - size/2
        kernel[i][j] = e**((-(x**2 + y**2))/(2*(sigma**2)))/(2*pi*(sigma**2))
    return kernel

Once you are done, compare the output of the simple kernel with the gaussian kernel.

In [ ]:
kernel = gaussian_kernel(3, 1)

cv_imshow(convolution_smooth(input, kernel))

## Challenge 2: critical thinking

If your code is working correctly, you see an image that is smoother, but also darker. What happened?

A operação de filtro gaussiano gera uma média que tem o peso dependendo da distância do centro da imagem. Ou seja, por ser uma média, os valores de cada píxel são diminuídos e assim o brilho também diminui.

You can click on the toc.png tab to the left to browse by section.